# 04wk-2: 감성분석 파고들기 (2)

최규빈  
2024-09-27

<a href="https://colab.research.google.com/github/guebin/MP2024/blob/main/posts/02wk-1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [1]:
# {{<video https://youtu.be/playlist?list=PLQqh36zP38-xZ2p62reZCCgZsyMQLsOqY&si=bHkFTktvJIGoD0ni >}}

# 2. Imports

In [2]:
import datasets
import transformers
import evaluate
import numpy as np
import torch # 파이토치

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

In [ ]:
datasets.load_dataset
datasets.Dataset.from_dict, 
datasets.dataset_dict.DatasetDict,
transformers.AutoTokenizer.from_pretrained,
transformers.AutoModelForSequenceClassification.from_pretrained,
transformers.DataCollatorWithPadding,
transformers.TrainingArguments,
transformers.Trainer,
transformers.pipeline,
evaluate.load

# 3. 이전시간 요약

`-` 이전시간의 내용중 이번시간에 기억할것들을 요약

`-` `DatasetDict`: 임의의 자료에 대한 `DatasetDict` 오브젝트 만들기

In [3]:
train_dict = {
    'text': [
        "I prefer making decisions based on logic and objective facts.",
        "I always consider how others might feel when making a decision.",
        "Data and analysis drive most of my decisions.",
        "I rely on my empathy and personal values to guide my choices."
    ],
    'label': [0, 1, 0, 1]  # 0은 T(사고형), 1은 F(감정형)
}

test_dict = {
    'text': [
        "I find it important to weigh all the pros and cons logically.",
        "When making decisions, I prioritize harmony and people's emotions."
    ],
    'label': [0, 1]  # 0은 T(사고형), 1은 F(감정형)
}
train_data = datasets.Dataset.from_dict(train_dict)
test_data = datasets.Dataset.from_dict(test_dict)
나의데이터 = datasets.dataset_dict.DatasetDict({'train':train_data, 'test':test_data})

`-` `DatasetDict`: 아래의 코드도 가능

In [4]:
나의데이터['train']['text']

`-` `토크나이저`: 토크나이저는 “`Str` $\to$ `Dict`” 인 함수이다.

In [5]:
데이터전처리하기1 = 토크나이저 = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") 

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(

In [6]:
토크나이저(나의데이터['train']['text'][0])

`-` `토크나이저`: 토크나이저의 “Str $\to$ Dict” 인 함수는 배치처리가
가능하다.

In [7]:
토크나이저(나의데이터['train']['text'])

`-` `인공지능`: 인공지능은 많은 파라메터를 포함하고 있는 어떠한
물체이다.

`-` `인공지능`: 인공지능의 파라메터는 변화할 수 있으며, loss가 더
작은쪽으로 파라메터를 변화시키는 과정을 “학습”이라고 부른다.

`-` `인공지능`: 인공지능은 “`**Dict` $\to$
`transformers.modeling_outputs.SequenceClassifierOutput`”인 함수이다.
그런데 쓰기 까다롭다.

-   `1`. `Dict`에는 특정한 key를 포함하고 있어야한다. (`input_ids`,
    `attention_mask`)
-   `2`. key에 대응하는 숫자는 파이토치 텐서형태이어야 한다. (`3`.
    따라서 (m,n)꼴의 차원을 **반드시** 가져야 한다)
-   `4`. `Dict`에 `labels`이 (텐서형으로) 포함된 경우 loss가 계산된다.
    (그리고 이걸 계산해야지 학습을 할 수 있음)

In [8]:
인공지능 = model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

`#` 입력예시1

In [9]:
인공지능(
    **{
        'input_ids': torch.tensor([[ 101, 1045, 102],[101, 9544, 102]]), 
        'attention_mask': torch.tensor([[1, 1, 1],[1, 1, 1]]) # 생략가능
    }
)
# 인공지능이 문제를 푸는 상황으로 비유할 수 있음

`#`

`#` 입력예시2

In [10]:
인공지능(
    **{
        'input_ids': torch.tensor([[ 101, 1045, 102],[101, 9544, 102]]), 
        'attention_mask': torch.tensor([[1, 1, 1],[1, 1, 1]]), # 생략가능
        'labels': torch.tensor([1, 0]) # 생략가능
    }
)
# 인공지능이 문제를 풀고 자기 스스로 채점까지 하는 상황으로 비유할 수 있음

`#`

`-` `인공지능`: 인공지능의 출력결과
“`transformers.modeling_outputs.SequenceClassifierOutput`” 에서
확률/예측값을 추출하는 방법

-   인공지능의 출력결과 $\to$ 로짓 $\to$ 확률 $\to$ 인공지능의예측
-   인공지능의 출력결과 $\to$ 로짓 $\to$ 인공지능의예측

In [11]:
계산된숫자들_로짓 = 인공지능(
    **{
        'input_ids': torch.tensor([[ 101, 1045, 102],[101, 9544, 102]]), 
        'attention_mask': torch.tensor([[1, 1, 1],[1, 1, 1]]), # 생략가능
        'labels': torch.tensor([1, 0]) # 생략가능
    }
).logits

로짓 $\to$ 확률

In [12]:
계산된숫자들_로짓

In [13]:
torch.exp(계산된숫자들_로짓) / torch.exp(계산된숫자들_로짓).sum(axis=1)

In [14]:
def 확률계산하기(인공지능출력):
    계산된숫자들_로짓 = 인공지능출력.logits
    return torch.exp(계산된숫자들_로짓) / torch.exp(계산된숫자들_로짓).sum(axis=1)

로짓 $\to$ 인공지능의예측

In [15]:
계산된숫자들_로짓.argmax(axis=1)

# 4. 데이터전처리하기2

`-` 아래코드를 파고들어보자.

``` python
def 데이터전처리하기2(examples):
    return 데이터전처리하기1(examples["text"], truncation=True)
전처리된나의데이터 = 나의데이터.map(lambda x: {'dummy': '메롱'})
```

`-` `examples['text']`의 쓰임으로 유추해본결과 examples에 가정된 입력의
형태??

In [16]:
train_data['text']

In [17]:
train_data[0]['text']

`-` 개념1: Hugging Face의 `datasets` 라이브러리에서 제공하는
`datasets.dataset_dict.DatasetDict`은, 요소들의 변환에 특화된
`map`이라는 메소드가(=함수가) 내장되어있다.

`# 예시1`

In [18]:
train_dict = {
    'text': [
        "I prefer making decisions based on logic and objective facts.",
        "I always consider how others might feel when making a decision.",
        "Data and analysis drive most of my decisions.",
        "I rely on my empathy and personal values to guide my choices."
    ],
    'label': [0, 1, 0, 1]  # 0은 T(사고형), 1은 F(감정형)
}

test_dict = {
    'text': [
        "I find it important to weigh all the pros and cons logically.",
        "When making decisions, I prioritize harmony and people's emotions."
    ],
    'label': [0, 1]  # 0은 T(사고형), 1은 F(감정형)
}
train_data = datasets.Dataset.from_dict(train_dict)
test_data = datasets.Dataset.from_dict(test_dict)
나의데이터 = datasets.dataset_dict.DatasetDict({'train':train_data, 'test':test_data})

In [19]:
datasets.dataset_dict.DatasetDict

In [20]:
datasets.dataset_dict.DatasetDict

::: {#cell-46 .cell execution_count=21}

``` python
나의데이터.map?
```

::: {.cell-output .cell-output-display}

<pre><span class="ansi-red-fg">Signature:</span>
나의데이터<span class="ansi-blue-fg">.</span>map<span class="ansi-blue-fg">(</span>
    function<span class="ansi-blue-fg">:</span> Optional<span class="ansi-blue-fg">[</span>Callable<span class="ansi-blue-fg">]</span> <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">None</span><span class="ansi-blue-fg">,</span>
    with_indices<span class="ansi-blue-fg">:</span> bool <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">False</span><span class="ansi-blue-fg">,</span>
    with_rank<span class="ansi-blue-fg">:</span> bool <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">False</span><span class="ansi-blue-fg">,</span>
    input_columns<span class="ansi-blue-fg">:</span> Union<span class="ansi-blue-fg">[</span>str<span class="ansi-blue-fg">,</span> List<span class="ansi-blue-fg">[</span>str<span class="ansi-blue-fg">]</span><span class="ansi-blue-fg">,</span> NoneType<span class="ansi-blue-fg">]</span> <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">None</span><span class="ansi-blue-fg">,</span>
    batched<span class="ansi-blue-fg">:</span> bool <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">False</span><span class="ansi-blue-fg">,</span>
    batch_size<span class="ansi-blue-fg">:</span> Optional<span class="ansi-blue-fg">[</span>int<span class="ansi-blue-fg">]</span> <span class="ansi-blue-fg">=</span> <span class="ansi-cyan-fg">1000</span><span class="ansi-blue-fg">,</span>
    drop_last_batch<span class="ansi-blue-fg">:</span> bool <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">False</span><span class="ansi-blue-fg">,</span>
    remove_columns<span class="ansi-blue-fg">:</span> Union<span class="ansi-blue-fg">[</span>str<span class="ansi-blue-fg">,</span> List<span class="ansi-blue-fg">[</span>str<span class="ansi-blue-fg">]</span><span class="ansi-blue-fg">,</span> NoneType<span class="ansi-blue-fg">]</span> <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">None</span><span class="ansi-blue-fg">,</span>
    keep_in_memory<span class="ansi-blue-fg">:</span> bool <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">False</span><span class="ansi-blue-fg">,</span>
    load_from_cache_file<span class="ansi-blue-fg">:</span> Optional<span class="ansi-blue-fg">[</span>bool<span class="ansi-blue-fg">]</span> <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">None</span><span class="ansi-blue-fg">,</span>
    cache_file_names<span class="ansi-blue-fg">:</span> Optional<span class="ansi-blue-fg">[</span>Dict<span class="ansi-blue-fg">[</span>str<span class="ansi-blue-fg">,</span> Optional<span class="ansi-blue-fg">[</span>str<span class="ansi-blue-fg">]</span><span class="ansi-blue-fg">]</span><span class="ansi-blue-fg">]</span> <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">None</span><span class="ansi-blue-fg">,</span>
    writer_batch_size<span class="ansi-blue-fg">:</span> Optional<span class="ansi-blue-fg">[</span>int<span class="ansi-blue-fg">]</span> <span class="ansi-blue-fg">=</span> <span class="ansi-cyan-fg">1000</span><span class="ansi-blue-fg">,</span>
    features<span class="ansi-blue-fg">:</span> Optional<span class="ansi-blue-fg">[</span>datasets<span class="ansi-blue-fg">.</span>features<span class="ansi-blue-fg">.</span>features<span class="ansi-blue-fg">.</span>Features<span class="ansi-blue-fg">]</span> <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">None</span><span class="ansi-blue-fg">,</span>
    disable_nullable<span class="ansi-blue-fg">:</span> bool <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">False</span><span class="ansi-blue-fg">,</span>
    fn_kwargs<span class="ansi-blue-fg">:</span> Optional<span class="ansi-blue-fg">[</span>dict<span class="ansi-blue-fg">]</span> <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">None</span><span class="ansi-blue-fg">,</span>
    num_proc<span class="ansi-blue-fg">:</span> Optional<span class="ansi-blue-fg">[</span>int<span class="ansi-blue-fg">]</span> <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">None</span><span class="ansi-blue-fg">,</span>
    desc<span class="ansi-blue-fg">:</span> Optional<span class="ansi-blue-fg">[</span>str<span class="ansi-blue-fg">]</span> <span class="ansi-blue-fg">=</span> <span class="ansi-green-fg">None</span><span class="ansi-blue-fg">,</span>
<span class="ansi-blue-fg">)</span> <span class="ansi-blue-fg">-&gt;</span> <span class="ansi-blue-fg">'DatasetDict'</span>
<span class="ansi-red-fg">Docstring:</span>
Apply a function to all the elements in the table (individually or in batches)
and update the table (if function does updated examples).
The transformation is applied to all the datasets of the dataset dictionary.
Args:
    function (`callable`): with one of the following signature:
        - `function(example: Dict[str, Any]) -&gt; Dict[str, Any]` if `batched=False` and `with_indices=False`
        - `function(example: Dict[str, Any], indices: int) -&gt; Dict[str, Any]` if `batched=False` and `with_indices=True`
        - `function(batch: Dict[str, List]) -&gt; Dict[str, List]` if `batched=True` and `with_indices=False`
        - `function(batch: Dict[str, List], indices: List[int]) -&gt; Dict[str, List]` if `batched=True` and `with_indices=True`
        For advanced usage, the function can also return a `pyarrow.Table`.
        Moreover if your function returns nothing (`None`), then `map` will run your function and return the dataset unchanged.
    with_indices (`bool`, defaults to `False`):
        Provide example indices to `function`. Note that in this case the signature of `function` should be `def function(example, idx): ...`.
    with_rank (`bool`, defaults to `False`):
        Provide process rank to `function`. Note that in this case the
        signature of `function` should be `def function(example[, idx], rank): ...`.
    input_columns (`[Union[str, List[str]]]`, *optional*, defaults to `None`):
        The columns to be passed into `function` as
        positional arguments. If `None`, a dict mapping to all formatted columns is passed as one argument.
    batched (`bool`, defaults to `False`):
        Provide batch of examples to `function`.
    batch_size (`int`, *optional*, defaults to `1000`):
        Number of examples per batch provided to `function` if `batched=True`,
        `batch_size &lt;= 0` or `batch_size == None` then provide the full dataset as a single batch to `function`.
    drop_last_batch (`bool`, defaults to `False`):
        Whether a last batch smaller than the batch_size should be
        dropped instead of being processed by the function.
    remove_columns (`[Union[str, List[str]]]`, *optional*, defaults to `None`):
        Remove a selection of columns while doing the mapping.
        Columns will be removed before updating the examples with the output of `function`, i.e. if `function` is adding
        columns with names in `remove_columns`, these columns will be kept.
    keep_in_memory (`bool`, defaults to `False`):
        Keep the dataset in memory instead of writing it to a cache file.
    load_from_cache_file (`Optional[bool]`, defaults to `True` if caching is enabled):
        If a cache file storing the current computation from `function`
        can be identified, use it instead of recomputing.
    cache_file_names (`[Dict[str, str]]`, *optional*, defaults to `None`):
        Provide the name of a path for the cache file. It is used to store the
        results of the computation instead of the automatically generated cache file name.
        You have to provide one `cache_file_name` per dataset in the dataset dictionary.
    writer_batch_size (`int`, default `1000`):
        Number of rows per write operation for the cache file writer.
        This value is a good trade-off between memory usage during the processing, and processing speed.
        Higher value makes the processing do fewer lookups, lower value consume less temporary memory while running `map`.
    features (`[datasets.Features]`, *optional*, defaults to `None`):
        Use a specific [`Features`] to store the cache file
        instead of the automatically generated one.
    disable_nullable (`bool`, defaults to `False`):
        Disallow null values in the table.
    fn_kwargs (`Dict`, *optional*, defaults to `None`):
        Keyword arguments to be passed to `function`
    num_proc (`int`, *optional*, defaults to `None`):
        Number of processes for multiprocessing. By default it doesn't
        use multiprocessing.
    desc (`str`, *optional*, defaults to `None`):
        Meaningful description to be displayed alongside with the progress bar while mapping examples.
Example:
```py
&gt;&gt;&gt; from datasets import load_dataset
&gt;&gt;&gt; ds = load_dataset("rotten_tomatoes")
&gt;&gt;&gt; def add_prefix(example):
...     example["text"] = "Review: " + example["text"]
...     return example
&gt;&gt;&gt; ds = ds.map(add_prefix)
&gt;&gt;&gt; ds["train"][0:3]["text"]
['Review: the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'Review: the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .',
 'Review: effective but too-tepid biopic']
# process a batch of examples
&gt;&gt;&gt; ds = ds.map(lambda example: tokenizer(example["text"]), batched=True)
# set number of processors
&gt;&gt;&gt; ds = ds.map(add_prefix, num_proc=4)

<span class="ansi-red-fg">File:</span>
~/anaconda3/envs/hf/lib/python3.12/site-packages/datasets/dataset_dict.py
<span class="ansi-red-fg">Type:</span> method

</pre>

    :::

    :::
    :::


    - 당장필요한내용: `datasets.dataset_dict.DatasetDict.map()`은 (1) `function`을 입력으로 받는데, (2) `function`은 Dict를 입력으로 받고 Dict를 출력하는 함수이어야 한다. 

    ::: {#cell-48 .cell execution_count=22}
    ``` {.python .cell-code}
    전처리된나의데이터 = 나의데이터.map(lambda x: {'dummy': '메롱'})

    Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 2347.12 examples/s]
    Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1523.54 examples/s]

In [23]:
나의데이터['train'][0], 전처리된나의데이터['train'][0]

In [24]:
for d in 전처리된나의데이터['train']:
    print(d)

{'text': 'I prefer making decisions based on logic and objective facts.', 'label': 0, 'dummy': '메롱'}
{'text': 'I always consider how others might feel when making a decision.', 'label': 1, 'dummy': '메롱'}
{'text': 'Data and analysis drive most of my decisions.', 'label': 0, 'dummy': '메롱'}
{'text': 'I rely on my empathy and personal values to guide my choices.', 'label': 1, 'dummy': '메롱'}

In [25]:
for d in 전처리된나의데이터['test']:
    print(d)

{'text': 'I find it important to weigh all the pros and cons logically.', 'label': 0, 'dummy': '메롱'}
{'text': "When making decisions, I prioritize harmony and people's emotions.", 'label': 1, 'dummy': '메롱'}

`#`

`# 예시2`

In [26]:
전처리된나의데이터 = 나의데이터.map(lambda x: {'dummy':'메롱', 'label2': '사고형(T)'} if x['label'] == 0 else {'dummy':'메롱','label2': '감정형(F)'})

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1539.19 examples/s]

In [27]:
for d in 전처리된나의데이터['train']:
    print(d)

{'text': 'I prefer making decisions based on logic and objective facts.', 'label': 0, 'dummy': '메롱', 'label2': '사고형(T)'}
{'text': 'I always consider how others might feel when making a decision.', 'label': 1, 'dummy': '메롱', 'label2': '감정형(F)'}
{'text': 'Data and analysis drive most of my decisions.', 'label': 0, 'dummy': '메롱', 'label2': '사고형(T)'}
{'text': 'I rely on my empathy and personal values to guide my choices.', 'label': 1, 'dummy': '메롱', 'label2': '감정형(F)'}

In [28]:
for d in 전처리된나의데이터['test']:
    print(d)

{'text': 'I find it important to weigh all the pros and cons logically.', 'label': 0, 'dummy': '메롱', 'label2': '사고형(T)'}
{'text': "When making decisions, I prioritize harmony and people's emotions.", 'label': 1, 'dummy': '메롱', 'label2': '감정형(F)'}

`#`

`# 예시3`

In [29]:
def 데이터전처리하기2(examples):
    return 데이터전처리하기1(examples["text"], truncation=True)

In [30]:
전처리된나의데이터 = 나의데이터.map(데이터전처리하기2,batched=True)

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1161.37 examples/s]

In [31]:
전처리된나의데이터['train'][0]

`#`

# 5. 데이터콜렉터

|                |    주어진자료     | $\overset{tokenizer,map}{\Longrightarrow}$ 전처리된자료 | $\overset{datacollector}{\Longrightarrow}$더전처리된자료 |
|:----------------:|:----------------:|:----------------:|:----------------:|
|  Dict의 Keys   |  `text`,`label`   |         `input_ids`, `attention_mask`, `label`          |         `input_ids`, `attention_mask`, `labels`          |
|   자료의형태   |    텍스트,라벨    |                   숫자화 O, 행렬화 X                    |                    숫자화 O, 행렬화 O                    |
| `torch.tensor` |        \-         |                            X                            |                            O                             |
|    미니배치    |        \-         |                            X                            |                            O                             |
| 패딩/동적패딩  |        \-         |                            X                            |                            O                             |
|    예측할때    | 강인공지능의 입력 |                     트레이너의 입력                     |                     인공지능의 입력                      |

`-` 데이터콜렉터 사용방법

In [32]:
데이터콜렉터 = transformers.DataCollatorWithPadding(tokenizer=토크나이저,return_tensors='pt')
데이터콜렉터?

-   당장필요한것: 입력은 `[딕셔너리, 딕셔너리, 딕셔너리, ...]` 의 형태

In [33]:
데이터콜렉터(
    [
        dict(label=0,input_ids=[101,1045,102],attention_mask=[1,1,1]),
        dict(label=0,input_ids=[101,1045,9544,102],attention_mask=[1,1,1,1]),
    ]
)

In [34]:
인공지능(**데이터콜렉터(
    [
        dict(label=0,input_ids=[101,1045,102],attention_mask=[1,1,1]),
        dict(label=0,input_ids=[101,1045,9544,102],attention_mask=[1,1,1,1]),
    ]
))

# 6. 평가하기

`-` `accuracy.compute`의 기능

In [35]:
accuracy = evaluate.load("accuracy")

In [36]:
accuracy.compute(references=[0, 0, 0], predictions=[0, 1, 1])

`-` 함수내용

In [37]:
# def 평가하기(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     accuracy = evaluate.load("accuracy")
#     return accuracy.compute(predictions=predictions, references=labels)

In [38]:
def 평가하기(eval_pred):
    계산된숫자들_로짓, 실제정답 = eval_pred
    인공지능의예측 = np.argmax(계산된숫자들_로짓, axis=1)
    accuracy = evaluate.load("accuracy")
    return accuracy.compute(predictions=인공지능의예측, references=실제정답)

# 7. 트레이너

## A. 트레이너의 제1역할 – CPU에서 GPU로..

`-` Step1~4 를 위한 준비

In [39]:
## Step1 
데이터불러오기 = datasets.load_dataset
데이터전처리하기1 = 토크나이저 = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") 
def 데이터전처리하기2(examples):
    return 데이터전처리하기1(examples["text"], truncation=True)
## Step2 
인공지능생성하기 = transformers.AutoModelForSequenceClassification.from_pretrained
## Step3 
데이터콜렉터 = transformers.DataCollatorWithPadding(tokenizer=토크나이저)
def 평가하기(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = evaluate.load("accuracy")
    return accuracy.compute(predictions=predictions, references=labels)
트레이너세부지침생성기 = transformers.TrainingArguments
트레이너생성기 = transformers.Trainer
## Step4 
강인공지능생성하기 = transformers.pipeline

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(

`-` Step1~2 까지의 코드 실행

In [40]:
데이터 = 데이터불러오기('imdb')
전처리된데이터 = 데이터.map(데이터전처리하기2,batched=True)
전처리된훈련자료, 전처리된검증자료 = 전처리된데이터['train'], 전처리된데이터['test']
## Step2 
torch.manual_seed(43052)
인공지능 = 인공지능생성하기("distilbert/distilbert-base-uncased", num_labels=2)

Using the latest cached version of the dataset since imdb couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at /home/cgb3/.cache/huggingface/datasets/imdb/plain_text/0.0.0/e6281661ce1c48d982bc483cf8a173c1bbeb5d31 (last modified on Thu Sep 26 23:35:50 2024).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

`-` 인공지능의 파라메터 상태확인 1

In [41]:
인공지능.classifier.weight

-   중요한내용1: 숫자들
-   중요한내용2: 나 CPU에 있어요..

`-` 인공지능을 이용한 예측 1

In [42]:
확률계산하기(인공지능(**데이터콜렉터([토크나이저("This movie was a huge disappointment.")])))

`-` 트레이너 생성: 생성되자마자 하는일 = 모델을 CPU에서 GPU로 옮긴다.

In [43]:
## Step3 
트레이너세부지침 = 트레이너세부지침생성기(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2, # 전체문제세트를 2번 공부하라..
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)
트레이너 = 트레이너생성기(
    model=인공지능,
    args=트레이너세부지침,
    train_dataset=전처리된훈련자료,
    eval_dataset=전처리된검증자료,
    tokenizer=토크나이저,
    data_collator=데이터콜렉터,
    compute_metrics=평가하기,
)
# 트레이너.train()
# ## Step4 
# 강인공지능 = 강인공지능생성하기("sentiment-analysis", model="my_awesome_model/checkpoint-1563")
# print(강인공지능("This movie was a huge disappointment."))
# print(강인공지능("This was a masterpiece."))

`-` 인공지능의 파라메터 상태확인 2

In [44]:
인공지능.classifier.weight

-   중요한내용1: 숫자들 (똑같나?)
-   중요한내용2: `device='cuda:0'`, (나 GPU에 있어요)

`-` 인공지능을 이용한 예측 2

In [45]:
# 확률계산하기(인공지능(**데이터콜렉터([토크나이저("This movie was a huge disappointment.")]))) # 에러발생

-   인공지능은 GPU에 있는데, 데이터는 CPU에 있음.. $\to$ 에러발생

In [46]:
확률계산하기(인공지능(**데이터콜렉터([토크나이저("This movie was a huge disappointment.")]).to("cuda:0")))

`-` 트레이너의 제1역할: 트레이너는 생성과 동시에 하는역할이 있는데, 바로
인공지능을 GPU에 올리는 것이다.

## B. 트레이너의 제2역할 – 예측하기

`-` 트레이너의 제2역할: `트레이너.predict()` 사용가능

-   `트레이너.predict()`의 입력형태는 input_ids, attention_mask, label
    이 존재하는 `Dataset`

In [47]:
sample_dict = {
    'text': ["This movie was a huge disappointment."],
    'label': [0],
    'input_ids': [[101, 2023, 3185, 2001, 1037, 4121, 10520, 1012, 102]],
    'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]
}
sample_dataset = datasets.Dataset.from_dict(sample_dict)
sample_dataset

In [48]:
트레이너.predict(sample_dataset)

In [49]:
logits = np.array([[-0.11731032,  0.02610314]])
np.exp(logits)/np.exp(logits).sum(axis=1)

아래의 결과와 동일

In [50]:
확률계산하기(인공지능(**데이터콜렉터([토크나이저("This movie was a huge disappointment.")]).to("cuda:0")))

`-` 이렇게 쓰면 좋음.

In [52]:
트레이너.predict(전처리된데이터['train'])

-   `train`에서 문제 품

In [53]:
트레이너.predict(전처리된데이터['test'])

-   `test`에서 문제 품

## C. 트레이너의 제3역할 – 학습 및 결과저장

In [54]:
트레이너.train()

In [55]:
25000 / 16 

In [56]:
1563 * 2 

`-` 인공지능이 똑똑해졌을까?

`-` 인공지능의 파라메터 상태확인 3

In [62]:
인공지능.classifier.weight # 진짜 살짝 바뀐것 같은데?

-   중요한내용1: 숫자들
-   중요한내용2: `device='cuda:0'`, (나 GPU에 있어요)

`-` 인공지능의 파라메터 상태확인 2와 비교삿

``` python
인공지능.classifier.weight
```

    Parameter containing:
    tensor([[-0.0234,  0.0279,  0.0242,  ...,  0.0091, -0.0063, -0.0133],
            [ 0.0087,  0.0007, -0.0099,  ...,  0.0183, -0.0007,  0.0295]],
           device='cuda:0', requires_grad=True)

`-` 숫자들이 바뀐걸 확인 $\to$ 뭔가 다른 계산결과를 준다는 의미겠지?
$\to$ 진짜 그런지 보자..

In [63]:
확률계산하기(인공지능(**데이터콜렉터([토크나이저("This movie was a huge disappointment.")]).to("cuda:0")))

In [64]:
확률계산하기(인공지능(**데이터콜렉터([토크나이저("This was a masterpiece.")]).to("cuda:0")))

`-` 우리가 가져야할 생각: 신기하다 X // 노가다 많이 했구나.. O

# 8. 파이프라인

`-` 강인공지능?

> ref: <https://zdnet.co.kr/view/?no=20160622145838>

In [66]:
강인공지능 = transformers.pipeline("sentiment-analysis", model="my_awesome_model/checkpoint-1563")
print(강인공지능("This movie was a huge disappointment."))
print(강인공지능("This was a masterpiece."))

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.

[{'label': 'LABEL_0', 'score': 0.9885253310203552}]
[{'label': 'LABEL_1', 'score': 0.978060781955719}]